# Turing Machines and Decidability

This notebook explores Turing machines, the halting problem, and the limits of computation using our automata library.

## Learning Objectives
- Understand Turing machine computation
- Explore decidable vs undecidable problems
- Visualize the halting problem
- Experience infinite loops in practice

In [ ]:
import sys
sys.path.append('../src')
from automata import TuringMachine
import time

## 1. Basic Turing Machine

Let's start with a simple TM that accepts strings ending with '1'.

In [ ]:
# TM that accepts strings ending with '1'
ends_with_1 = TuringMachine(
    states={'q0', 'q1', 'qaccept', 'qreject'},
    alphabet={'0', '1'},
    tape_alphabet={'0', '1', '_'},
    transitions={
        ('q0', '0'): ('q0', '0', 'R'),
        ('q0', '1'): ('q1', '1', 'R'),
        ('q0', '_'): ('qreject', '_', 'L'),
        ('q1', '0'): ('q0', '0', 'R'),
        ('q1', '1'): ('q1', '1', 'R'),
        ('q1', '_'): ('qaccept', '_', 'L')
    },
    start_state='q0',
    accept_states={'qaccept'},
    reject_states={'qreject'}
)

# Test the machine
test_strings = ['1', '01', '101', '0', '10', '']
for s in test_strings:
    result = ends_with_1.accepts(s)
    print(f"'{s}' -> {'ACCEPT' if result else 'REJECT'}")

## 2. Turing Machine with Trace

Let's watch a TM execute step by step.

In [ ]:
# Watch the TM process "101" with trace
print("Tracing execution of '101':")
print("=" * 40)
result, steps, trace = ends_with_1.run('101', trace=True)
print(f"\nFinal result: {'ACCEPT' if result else 'REJECT'}")
print(f"Total steps: {steps}")

## 3. Binary Increment Machine

A TM that adds 1 to a binary number.

In [ ]:
# TM that increments a binary number
binary_increment = TuringMachine(
    states={'q0', 'q1', 'q2', 'qaccept'},
    alphabet={'0', '1'},
    tape_alphabet={'0', '1', '_'},
    transitions={
        # Go to rightmost bit
        ('q0', '0'): ('q0', '0', 'R'),
        ('q0', '1'): ('q0', '1', 'R'),
        ('q0', '_'): ('q1', '_', 'L'),
        
        # Add 1 (carry propagation)
        ('q1', '0'): ('qaccept', '1', 'L'),  # 0+1=1, done
        ('q1', '1'): ('q1', '0', 'L'),       # 1+1=0, carry
        ('q1', '_'): ('qaccept', '1', 'R')   # Carry to new bit
    },
    start_state='q0',
    accept_states={'qaccept'}
)

# Test binary increment
test_cases = ['0', '1', '10', '11', '101', '111']
for binary in test_cases:
    result, steps, trace = binary_increment.run(binary)
    # Extract result from final tape
    if trace:
        final_tape = ''.join(trace[-1]['tape']).strip('_')
        print(f"{binary} + 1 = {final_tape} (decimal: {int(binary, 2)} + 1 = {int(final_tape, 2) if final_tape else 0})")
    else:
        print(f"{binary} -> processed in {steps} steps")

## 4. The Halting Problem

Let's create a TM that demonstrates the halting problem by entering an infinite loop.

In [ ]:
# TM that loops forever on certain inputs
infinite_loop_tm = TuringMachine(
    states={'q0', 'q1', 'qaccept'},
    alphabet={'0', '1'},
    tape_alphabet={'0', '1', 'X', '_'},
    transitions={
        # Accept empty string
        ('q0', '_'): ('qaccept', '_', 'R'),
        
        # Loop forever on '1'
        ('q0', '1'): ('q1', 'X', 'R'),
        ('q1', '1'): ('q1', '1', 'R'),
        ('q1', '_'): ('q0', '_', 'L'),
        ('q0', 'X'): ('q0', '1', 'R'),
        
        # Accept strings starting with '0'
        ('q0', '0'): ('qaccept', '0', 'R')
    },
    start_state='q0',
    accept_states={'qaccept'}
)

# Test with different inputs
print("Testing halting behavior:")
print("Empty string:")
result, steps, _ = infinite_loop_tm.run('', max_steps=10)
print(f"Result: {'ACCEPT' if result else 'REJECT/TIMEOUT'}, Steps: {steps}")

print("\nString '0':")
result, steps, _ = infinite_loop_tm.run('0', max_steps=10)
print(f"Result: {'ACCEPT' if result else 'REJECT/TIMEOUT'}, Steps: {steps}")

print("\nString '1' (should loop):")
result, steps, _ = infinite_loop_tm.run('1', max_steps=20, trace=False)
print(f"Result: {'ACCEPT' if result else 'REJECT/TIMEOUT'}, Steps: {steps}")
print("Note: This would run forever without max_steps limit!")

## 5. Visualizing the Infinite Loop

Let's watch the infinite loop in action with a trace.

In [ ]:
print("Watching infinite loop with trace (first 15 steps):")
print("=" * 50)
result, steps, trace = infinite_loop_tm.run('1', max_steps=15, trace=True)
print(f"\nStopped after {steps} steps - would continue forever!")

## 6. Decidable vs Undecidable Problems

Let's explore the concept of decidability with practical examples.

In [ ]:
# Decidable problem: Check if binary string represents even number
even_checker = TuringMachine(
    states={'q0', 'qaccept', 'qreject'},
    alphabet={'0', '1'},
    tape_alphabet={'0', '1', '_'},
    transitions={
        ('q0', '0'): ('q0', '0', 'R'),
        ('q0', '1'): ('q0', '1', 'R'),
        ('q0', '_'): ('q0', '_', 'L'),  # Go back to last bit
        ('q0', '0'): ('qaccept', '0', 'R'),  # Even if ends with 0
        ('q0', '1'): ('qreject', '1', 'R')   # Odd if ends with 1
    },
    start_state='q0',
    accept_states={'qaccept'},
    reject_states={'qreject'}
)

# This is actually simpler - just check last bit
simple_even_checker = TuringMachine(
    states={'q0', 'q1', 'qaccept', 'qreject'},
    alphabet={'0', '1'},
    tape_alphabet={'0', '1', '_'},
    transitions={
        ('q0', '0'): ('q1', '0', 'R'),
        ('q0', '1'): ('q0', '1', 'R'),
        ('q0', '_'): ('qaccept', '_', 'L'),  # Empty = 0 = even
        ('q1', '0'): ('q1', '0', 'R'),
        ('q1', '1'): ('q0', '1', 'R'),
        ('q1', '_'): ('qaccept', '_', 'L')   # Last bit was 0 = even
    },
    start_state='q0',
    accept_states={'qaccept'},
    reject_states={'qreject'}
)

print("Testing decidable problem (even number checker):")
test_numbers = ['0', '1', '10', '11', '100', '101', '110', '111']
for binary in test_numbers:
    decimal = int(binary, 2)
    result = simple_even_checker.accepts(binary)
    expected = (decimal % 2 == 0)
    status = "✓" if result == expected else "✗"
    print(f"{binary} (decimal {decimal}): {'EVEN' if result else 'ODD'} {status}")

## 7. The Halting Problem Demonstration

We cannot write a TM that decides if another TM halts, but we can demonstrate why.

In [ ]:
def simulate_halting_oracle(tm_description, input_string, max_steps=100):
    """
    Attempt to decide if a TM halts (this is impossible in general!)
    We can only give partial answers within time limits.
    """
    try:
        result, steps, _ = tm_description.run(input_string, max_steps=max_steps)
        if steps < max_steps:
            return f"HALTS (in {steps} steps)"
        else:
            return f"UNKNOWN (exceeded {max_steps} steps - might loop)"
    except Exception as e:
        return f"ERROR: {e}"

print("Attempting to solve the halting problem (impossible!):")
print("=" * 55)

# Test our "oracle" on different machines
test_cases = [
    (ends_with_1, '101', "TM that checks if string ends with 1"),
    (binary_increment, '111', "TM that increments binary number"),
    (infinite_loop_tm, '', "TM on empty string (should halt)"),
    (infinite_loop_tm, '1', "TM on '1' (infinite loop!)")
]

for tm, input_str, description in test_cases:
    result = simulate_halting_oracle(tm, input_str, max_steps=50)
    print(f"{description}")
    print(f"  Input: '{input_str}' -> {result}")
    print()

## 8. Rice's Theorem Illustration

Any non-trivial property of TM languages is undecidable.

In [ ]:
# Create TMs with different language properties

# TM that accepts everything
accept_all = TuringMachine(
    states={'q0', 'qaccept'},
    alphabet={'0', '1'},
    tape_alphabet={'0', '1', '_'},
    transitions={
        ('q0', '0'): ('qaccept', '0', 'R'),
        ('q0', '1'): ('qaccept', '1', 'R'),
        ('q0', '_'): ('qaccept', '_', 'R')
    },
    start_state='q0',
    accept_states={'qaccept'}
)

# TM that accepts nothing
accept_nothing = TuringMachine(
    states={'q0', 'qreject'},
    alphabet={'0', '1'},
    tape_alphabet={'0', '1', '_'},
    transitions={
        ('q0', '0'): ('qreject', '0', 'R'),
        ('q0', '1'): ('qreject', '1', 'R'),
        ('q0', '_'): ('qreject', '_', 'R')
    },
    start_state='q0',
    accept_states=set(),
    reject_states={'qreject'}
)

def test_language_property(tm, property_name, test_strings):
    """Test if a TM has a certain language property"""
    results = []
    for s in test_strings:
        try:
            accepts = tm.accepts(s, max_steps=10)
            results.append(accepts)
        except:
            results.append(None)  # Timeout/error
    return results

print("Rice's Theorem: Properties of TM languages are undecidable")
print("=" * 60)

test_strings = ['', '0', '1', '01', '10']
machines = [
    (accept_all, "Accepts everything"),
    (accept_nothing, "Accepts nothing"),
    (ends_with_1, "Accepts strings ending with 1"),
    (infinite_loop_tm, "May loop on some inputs")
]

for tm, description in machines:
    results = test_language_property(tm, description, test_strings)
    print(f"{description}:")
    for i, s in enumerate(test_strings):
        result = results[i]
        if result is None:
            print(f"  '{s}' -> TIMEOUT")
        else:
            print(f"  '{s}' -> {'ACCEPT' if result else 'REJECT'}")
    print()

print("Note: We cannot algorithmically determine if two TMs accept the same language!")

## 9. Conclusion

### Key Takeaways:

1. **Turing Machines** are the most powerful computational model
2. **The Halting Problem** is undecidable - no algorithm can determine if an arbitrary TM halts
3. **Rice's Theorem** shows that any non-trivial property of TM languages is undecidable
4. **Infinite loops** demonstrate the limits of computation
5. **Decidable problems** have algorithms that always halt with correct answers

### Practical Implications:
- Static analysis tools have fundamental limitations
- Perfect program verification is impossible
- Some problems require heuristics rather than algorithms
- Computational complexity theory emerges from these limits

### Next Steps:
- Explore complexity classes (P, NP, PSPACE)
- Study reduction techniques
- Investigate approximation algorithms
- Learn about quantum computation